In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

In [ ]:
# MediaPipe initialization
mp_pose = mp.solutions.pose
mp_face = mp.solutions.face_detection
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Video opening
cap = cv2.VideoCapture("videos_ashih/ashih2.mp4.mp4")
if not cap.isOpened():
    print("Video not opened")
    exit()

# Video parameters
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Prepare saving
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out = cv2.VideoWriter('videos_ashih/ashih2_processed.mp4', fourcc, fps, (width, height))

# Models initialization
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Main cycle
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pose_results = pose.process(frame_rgb)
    face_results = face.process(frame_rgb)
    hand_results = hands.process(frame_rgb)

    annotated_frame = frame.copy()

    if pose_results.pose_landmarks:
        mp_drawing.draw_landmarks(annotated_frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    if face_results.detections:
        for detection in face_results.detections:
            mp_drawing.draw_detection(annotated_frame, detection)

    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(annotated_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Show and save video
    cv2.imshow("Video processing", annotated_frame)
    out.write(annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Completion
cap.release()
out.release()
cv2.destroyAllWindows()
